In [26]:
from IPython import get_ipython
ipython = get_ipython()
ipython.magic('load_ext autoreload')
ipython.magic('autoreload 2')

import os
import sys
import time
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logging
sys.path.append('../../..')

from biodl.deephd.dhd_data import DHDDataset, load_config
from biodl.deephd.dhd_model import ASPPResNetSE
from biodl.deephd.dhd_losses import build_loss_by_name
from biodl.deephd.dhd_pipeline import DeepHDPipeline, _preprocess_batch
#
import torch
from torch.utils.data import DataLoader
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.logging import TensorBoardLogger

logging.basicConfig(level=logging.INFO)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
# path_cfg = '/mnt/data1T/data/annaha/homod/cfg.json'
# path_cfg = '/mnt/data1T/data/annaha/homod/cfg-lx.json'
path_cfg = '/mnt/data1T/data/annaha/homod/cfg-ce2.json'
#
cfg = load_config(path_cfg)
pipeline = DeepHDPipeline(path_cfg, num_workers=1)#.build()
# checkpoint_callback = ModelCheckpoint(filepath=os.path.join(pipeline.path_model, 'results'), verbose=True, monitor='val_loss', mode='min')
# logger = TensorBoardLogger(save_dir=pipeline.path_model, version=1)

In [28]:
# weights_path = glob.glob(os.path.join(logger.experiment.get_logdir(), '../../*.ckpt'))[0]
weights_path = glob.glob(os.path.join(pipeline.path_model, '*.ckpt'))[0]
# pipeline.load_state_dict(torch.load(weights_path, map_location=torch.device('cpu'))['state_dict'])
pipeline.load_state_dict(torch.load(weights_path)['state_dict'])
model = pipeline.model.to('cuda:0')
# dataloader_ = pipeline.val_dataloader()[0]
pipeline.eval()

, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace=True)
            (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          )
          (denselayer12): _DenseLayer(
            (norm1): BatchNorm2d(608, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace=True)
            (conv1): Conv2d(608, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace=True)
            (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          )
          (denselayer13): _DenseLayer(
            (norm1): BatchNorm2d(640, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inpla

In [4]:
dataloader_ = DataLoader(DHDDataset(cfg['val_abs'],
                                    crop_size=cfg['crop_size'],
                                    use_sasa=cfg['data']['use_sasa'],
                                    sasa_radiuses=cfg['data']['sasa_rad'],
                                    res_types=cfg['data']['res_types'],
                                    use_indices=cfg['data']['use_indices'],
                                    params_aug=cfg['aug'],
                                    dst_contact=cfg['data']['dst_contact'],
                                    test_mode=True).build(),
                        num_workers=4, batch_size=1)
len_dataloader = len(dataloader_)
step_ = int(np.ceil(len_dataloader / 10))
t1 = time.time()


INFO:root:	::load dataset into memory, #samples = 2634
INFO:root::: start (load-dataset) #data/#threads = 2634/1, use-process(True)
INFO:root:	[0/2634] (l1) (load-dataset)
INFO:root:	[527/2634] (l1) (load-dataset)
INFO:root:	[1054/2634] (l1) (load-dataset)
INFO:root:	[1581/2634] (l1) (load-dataset)
INFO:root:	[2108/2634] (l1) (load-dataset)
INFO:root:		... done, dt ~ 51.25 (s), speed=51.40 (samples/s) [load-dataset]
INFO:root:			... done, dt ~ 52.83 (s), #samples=2542 with size >= 64


In [29]:
dataset_iter = iter(dataloader_.dataset)

In [30]:
with torch.no_grad():
    x = next(dataset_iter)
    x_inp = torch.from_numpy(x['inp']).unsqueeze(dim=0).to('cuda:0').type(torch.float32)
    x_inp_npy = x_inp.cpu().numpy()
    #
    y_gt_npy = x['out'].astype(np.float32).squeeze()
    y_gt = torch.from_numpy(x['out']).unsqueeze(dim=0).to('cuda:0').type(torch.float32)
    y_pr_li = model.forward(x_inp)
    if not cfg['loss'].startswith('l'):
        y_pr = torch.sigmoid(y_pr_li)
    else:
        y_pr = y_pr_li
    y_pr_npy = y_pr.cpu().numpy()[0].squeeze()
    #
    loss_ = pipeline.trn_loss(y_pr_li, y_gt)
    #
    dst_t = 2.6
    plt.figure(figsize=(20, 8))
    plt.subplot(1, 5, 1)
    plt.imshow(y_pr_npy)
    plt.title('PR: max = {:0.1f}, loss={:0.2f}'.format(y_pr_npy.max(), float(loss_)))
    plt.subplot(1, 5, 2)
    plt.imshow(y_pr_npy > 0.5)
    plt.title(f'PR/T (<{dst_t})')
    plt.subplot(1, 5, 3)
    plt.imshow(y_gt_npy)
    plt.title('GT')
    plt.subplot(1, 5, 4)
    plt.imshow(y_gt_npy < dst_t)
    plt.title(f'GT/T (< {dst_t})')
    plt.subplot(1, 5, 5)
    plt.imshow(x_inp_npy[0, 0])
    plt.title('INP')
    plt.show()
    print('-')

RuntimeError: Given groups=1, weight of size 64 50 7 7, expected input[1, 48, 96, 96] to have 50 channels, but got 48 channels instead

In [10]:
y_gt.max()

tensor(1., device='cuda:0')

In [24]:
pipeline.trn_loss()

0.6411675347222222

In [36]:
with torch.no_grad():
    q = pipeline.trn_loss(y_pr, y_gt)



In [13]:
y_gt.mean()

tensor(0.0020, device='cuda:0')